# Implementación del Problema de Optimización de Carteras con Computador Cuántico Adiabático

Este cuaderno describe la implementación de un problema de optimización de carteras de inversiones utilizando un computador cuántico adiabático. El enfoque adoptado es el del "Minimum Volatility Portfolio", utilizando variables binarias para la selección de activos. El objetivo principal es minimizar el riesgo total de la cartera, modelado por la fórmula:

$$\min \sum_{i,j} x_i \cdot w_{ij} \cdot x_j$$

donde $x_i$ representa la inclusión (1) o no (0) del activo $i$ en la cartera, y $w_{ij}$ es el término de covarianza entre los activos $i$ y $j$.

Además, se establece una restricción sobre el retorno esperado de la cartera, asegurando que sea al menos igual a un umbral predefinido. Esto se modela con una variable slack y se expresa como:

$$R \leq \sum_{i} x_i \cdot r_i$$

donde $r_i$ es el retorno esperado del activo $i$.

Finalmente, como condición adicional, se determina que solo se quiere invertir en la mitad de las empresas disponibles. Esto se traduce en la restricción:

$$\sum_{i} x_i = \frac{N}{2}$$

donde $N$ es el número total de activos disponibles.



In [1]:
!pip install -r "requirements_unix.txt"

Processing ./dadk_light_3.10.tar.bz2
  Preparing metadata (setup.py) ... done
  Created wheel for dadk: filename=dadk-2023.12.10-py3-none-any.whl size=4995459 sha256=2c2a417721a622cd4e64fc7fa85dfda9c21f51c387683de343b0898aa97546fc
  Stored in directory: /tmp/pip-ephem-wheel-cache-7btzx8pk/wheels/78/36/68/08dcbec0b48f137a33fc3ac474d3838f74984d2dda7e3178dd
Successfully built dadk
  Attempting uninstall: dadk
    Found existing installation: dadk 2023.12.10
    Uninstalling dadk-2023.12.10:
      Successfully uninstalled dadk-2023.12.10


In [2]:
%matplotlib widget
from IPython.display import display, HTML
display(HTML("<style>.container{width:100% !important;}</style>"))
import random
from IPython.display import display, HTML
from dadk.Optimizer import *
from dadk.SolverFactory import *
from dadk.Solution_SolutionList import *
from dadk.BinPol import *
from random import uniform
from tabulate import tabulate
from numpy import argmax
import numpy as np
from dadk.QUBOSolverCPU import *
import pandas as pd
import numpy as np

In [48]:
# Cargar los datos del dataset
acciones = pd.read_csv('Dataset_Acciones_Pequenio.csv')

# Calcular los retornos diarios y la matriz de covarianza
# La función pct_change() calcula el cambio porcentual entre filas consecutivas, días en este caso
retornos = acciones.pct_change().dropna()
cov_matrix = retornos.cov().values

# Calcular los retornos esperados como la media de los retornos diarios (para la variable slack)
retornos_esperados = retornos.mean().values

# Obtener cantidad de activos N basado en las columnas seleccionadas
n_assets = len(acciones.columns)
asset_names = acciones.columns.tolist()

# Establecer un umbral de retorno esperado mínimo para la cartera (para la variable slack)
retorno_minimo = 0.02  # 2%

# Función para calcular el valor de stop para la variable slack
def calculate_slack_stop(retornos, retorno_min_esperado):
    # Calcular la suma máxima de los retornos
    max_returns_sum = sum(sorted(retornos, reverse=True)[:len(retornos) // 2])
    # Calcular el valor de stop para S
    slack_stop = max_returns_sum - retorno_min_esperado
    return slack_stop

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty, retornos, retorno_min_esperado):
    var_problema = BitArrayShape('x', (n_assets,))

    # Calcular el valor de stop para la variable slack
    slack_stop = calculate_slack_stop(retornos, retorno_min_esperado)

    # Crear la variable slack
    my_var_slack = VarSlack(name='slack_variable',start=0,step=1,stop=slack_stop, slack_type=SlackType.binary)

    var_shape_set = VarShapeSet(var_problema, my_var_slack)
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()

    # Construir H_cuad utilizando la matriz de covarianza
    for i in range(n_assets):
        for j in range(n_assets):
            w_ij = cov_matrix[i, j]
            H_cuad.add_term(w_ij, ('x', i), ('x', j))

    # Construir H_slack para representar la restricción del retorno mínimo esperado
    H_slack = BinPol()

    for i in range(n_assets):
        H_slack.add_term(retornos_esperados[i], ('x', i)) # ANTES PONÍA SOLO RETORNOS

    H_slack.add_slack_variable('slack_variable', factor=-1)
    H_slack.add_term(-retorno_minimo, ()) # ANTES PONIA EPSERADO

  # Construir H_half para representar la restricción de invertir en la mitad de empresas
    H_half = BinPol()

    for i in range(n_assets):
        H_half.add_term(1, ('x', i))
    H_half.add_term(-n_assets // 2, ())
    H_half.power(2)

    # Combinar los términos para formar el QUBO
    QUBO = H_cuad + (H_half + H_slack) * factor_penalty

    return QUBO, H_half


# Coeficiente de penalización para la restricción H_half
factor_penalty = 100
QUBO, H_half = build_qubo(cov_matrix, n_assets, factor_penalty, retornos_esperados, retorno_minimo)

# Configurar y ejecutar el solucionador QUBO
solver = QUBOSolverCPU(
    number_iterations=50000,
    number_runs=10, # Número de ejecuciones en paralelo
    scaling_bit_precision=16,
    auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING
)

solution_list = solver.minimize(QUBO)
print(solution_list.min_solution.configuration)
print()
print(solution_list.solver_times)


********************************************************************************
Scaling qubo, temperature_start, temperature_end and offset_increase_rate
  factor:                          108.00000
********************************************************************************


********************************************************************************
Effective values (including scaling factor)
  temperature_start:               28430.000
  temperature_end:                   114.600
  offset_increase_rate:             1081.000
  duration:                            0.001 sec
********************************************************************************

[1, 0, 0, 1]

+--------------+----------------------------+----------------------------+----------------+
| time         | from                       | to                         | duration       |
|--------------+----------------------------+----------------------------+----------------|
| anneal       | 2024-06-22 15:57:5

In [50]:
# Función para preparar y presentar el resultado
def prep_result(solution_list, H_half, silent=False):
    solution = solution_list.min_solution # La mejor solución
    constraint_penalty = H_half.compute(solution.configuration)
    if not silent:
        print(f'Valor QUBO: {constraint_penalty}')
        print(solution.configuration)
    return constraint_penalty, solution

# Función para reportar la solución
def report(constraint_penalty, solution, asset_names, retorno_min_esperado, silent=False):
    if not silent:
        if constraint_penalty == 0.0: # Es cero en las soluciones válidas
            print('Portfolio elegido:')
            # Asegurar que los índices no estén fuera del rango de asset_names
            selected_assets = [asset_names[i] for i, bit in enumerate(solution.configuration[:len(asset_names)]) if bit > 0.5]
            print(selected_assets)
        else:
            print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_min_esperado}")

# Función para verificar si el retorno mínimo es alcanzable
def es_factible(retornos, retorno_min_esperado):
    retorno_posible_max = sum(sorted(retornos, reverse=True)[:len(retornos) // 2])
    return retorno_posible_max >= retorno_min_esperado

# Verificar si el retorno mínimo es alcanzable antes de continuar
if not es_factible(retornos_esperados, retorno_minimo):
    print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_minimo}")
else:
    # Utilizar las funciones de resultado y reporte
    constraint_penalty, solution = prep_result(solution_list, H_half)
    report(constraint_penalty, solution, asset_names, retorno_minimo)

Valor QUBO: 0
[1, 0, 0, 1]
Portfolio elegido:
['EmpresaA', 'EmpresaD']


In [51]:
# Función para calcular la media y la desviación estándar de cada empresa
def calcular_metricas(df):
    metricas = pd.DataFrame(index=df.columns, columns=['Media', '  Desviación Estándar'])
    for column in df.columns:
        metricas.loc[column, 'Media'] = df[column].mean()
        metricas.loc[column, '  Desviación Estándar'] = df[column].std()
    return metricas

# Mostrar estadísticas de cada empresa
estadisticas = calcular_metricas(acciones)
print(estadisticas)

          Media   Desviación Estándar
EmpresaA   95.9             13.963842
EmpresaB   73.9             12.077987
EmpresaC   32.9              7.489993
EmpresaD  122.1             12.031902


Tras analizar el rendimiento de las distintas empresas así como la volatilidad de los precios de sus acciones, representados por la media y la desviación estándar respectivamente, se obtiene que las mejores empresas para invertir son la EmpresaD y la EmpresaA puesto que presentan un equilibrio favorable entre el potencial de rendimiento y el nivel de riesgo asociado.

La EmpresaD, con una media de 122.1 y una desviación estándar de 12.03, ofrece el mayor rendimiento de todas las opciones evaluadas, lo que indica una fuerte capacidad de generación de valor, manteniendo al mismo tiempo una volatilidad moderada que mitiga el riesgo de inversión.

Por su parte, la EmpresaA, aunque exhibe una desviación estándar ligeramente mayor de 13.96, también muestra un rendimiento robusto con una media de 95.9. Esta combinación sugiere que, a pesar de una mayor variabilidad en el precio de sus acciones, la EmpresaA sigue siendo una opción atractiva debido a su alta capacidad de retorno. Por lo tanto, seleccionar estas dos empresas para la inversión proporciona una diversificación estratégica que busca maximizar los beneficios ajustados por riesgo, aprovechando tanto la estabilidad relativa como el alto potencial de crecimiento.

In [52]:
import numpy as np
import pandas as pd

def evaluate_portfolio(solution, asset_names, retornos, cov_matrix, annual_risk_free_rate=0.01, retorno_del_mercado=0.10):
    pesos = np.array(solution.configuration, dtype=float)

    optimal_portfolio = solution_list.min_solution.configuration
    # Calcular el retorno y la varianza del portfolio
    retorno = sum(retorno * x for retorno, x in zip(retornos_esperados, optimal_portfolio)) * 252 *0.01
    #retorno = np.dot(pesos, retornos.mean())
    varianza = np.dot(pesos.T, np.dot(cov_matrix, pesos))

    # Convertir la tasa libre de riesgo anual a diaria
    daily_risk_free_rate = (1 + annual_risk_free_rate)**(1/252) - 1

    # Convertir los pesos de la solución a tipo flotante y normalizarlos
    pesos /= pesos.sum()

    # Calcular el retorno y la varianza del portfolio
    retorno_portfolio = np.dot(pesos, retornos.mean())
    varianza_portfolio = np.dot(pesos.T, np.dot(cov_matrix, pesos))
    desviacion_portfolio = np.sqrt(varianza_portfolio)

    # Calcular el ratio de Sharpe
    ratio_sharpe = (retorno_portfolio - daily_risk_free_rate) / desviacion_portfolio if desviacion_portfolio > 0 else 0

    # Identificar retornos negativos y calcular el Ratio de Sortino
    media_retornos = retornos.mean()
    retornos_negativos = media_retornos < daily_risk_free_rate
    indices_negativos = np.where(retornos_negativos)[0]
    if len(indices_negativos) > 0:
        pesos_negativos = pesos[indices_negativos]
        covarianza_negativa = cov_matrix[np.ix_(indices_negativos, indices_negativos)]
        varianza_negativa = np.dot(pesos_negativos.T, np.dot(covarianza_negativa, pesos_negativos))
        desviacion_estandar_negativos = np.sqrt(varianza_negativa)
        ratio_sortino = (retorno_portfolio - daily_risk_free_rate) / desviacion_estandar_negativos if desviacion_estandar_negativos > 0 else 0
    else:
        ratio_sortino = float('inf')  # Infinito si no hay retornos negativos

    # Calcular el ratio de Información
    exceso_retornos = retorno_portfolio - retorno_del_mercado
    desviacion_estandar_exceso = desviacion_portfolio
    ratio_informacion = exceso_retornos / desviacion_estandar_exceso if desviacion_estandar_exceso > 0 else 0

    # Calcular la máxima caída
    precios_portfolio = np.cumprod(1 + retornos.mean())
    wealth_index = 1000 * precios_portfolio
    previous_peaks = np.maximum.accumulate(wealth_index)
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    max_drawdown = drawdowns.min()

    # Imprimir las métricas
    print(f"Retorno del Portfolio: {retorno}")
    print(f"Varianza del Portfolio: {varianza}")
    print(f"Ratio de Sharpe: {ratio_sharpe}")
    print(f"Ratio de Sortino: {ratio_sortino}")
    print(f"Ratio de Información: {ratio_informacion}")
    print(f"Máxima Caída: {max_drawdown}")

# Llamar a la función de evaluación después de obtener la solución
if solution_list.min_solution:
    evaluate_portfolio(solution_list.min_solution, asset_names, retornos, cov_matrix)


Retorno del Portfolio: 0.02119485042884988
Varianza del Portfolio: 0.049128903525671064
Ratio de Sharpe: 0.03758929457817526
Ratio de Sortino: 0.0664725431289585
Ratio de Información: -0.8643762332153453
Máxima Caída: -0.016377457564061806


**Retorno del Portfolio**
-  0.02119 (2.119%): Este es un retorno positivo, lo que indica que el portafolio ha generado ganancias sobre el periodo de tiempo considerado. Es superior al retorno mínimo establecido del 2%. En términos generales, este es un buen signo, aunque la evaluación de si es un buen retorno también depende del contexto del mercado y del nivel de riesgo asociado.

**Varianza del Portfolio**
-  0.04913 (4.913%): La varianza del portafolio indica la dispersión de los retornos alrededor de su media. Una varianza más alta sugiere un mayor nivel de riesgo. En este caso, una varianza de aproximadamente 0.049 puede considerarse moderadamente alta, dependiendo de la tolerancia al riesgo del inversor y de las comparaciones con benchmarks relevantes.

**Ratio de Sharpe**
-  0.03759 (3.759%): El Ratio de Sharpe es positivo, lo que es favorable ya que indica que el portafolio está generando un exceso de retorno sobre la tasa libre de riesgo, ajustado por volatilidad. Sin embargo, este valor es relativamente bajo, lo que sugiere que el exceso de retorno ajustado por el riesgo es modesto. Un Ratio de Sharpe superior a 1 es generalmente considerado muy bueno, entre 0.5 y 1 es aceptable, y menos de 0.5 puede considerarse bajo dependiendo del contexto.

**Ratio de Sortino**
-  0.06647: Este valor es superior al Ratio de Sharpe, lo cual es esperado ya que el Ratio de Sortino solo considera la volatilidad negativa. Un valor mayor aquí indica que el portafolio maneja bien las caídas, proporcionando un mejor retorno ajustado al riesgo a la baja. Esto es particularmente importante para los inversores que son sensibles a las pérdidas.

**Ratio de Información**
-  -0.86438: Este ratio es negativo y considerablemente bajo, lo que indica que el portafolio está sub-rendimiento en comparación con el benchmark (mercado). Este es un punto de preocupación, ya que sugiere que la gestión del portafolio no está agregando valor en términos de exceso de retorno sobre el mercado, ajustado por el riesgo del exceso de retorno.

**Máxima Caída**
-  -0.01638: La máxima caída es una medida de la pérdida máxima que se habría experimentado si se hubiera comprado en el pico más alto y vendido en el valle más bajo antes de una nueva recuperación. Una caída máxima de aproximadamente -1.64% es relativamente baja, lo que sugiere que el portafolio tiene un buen control del riesgo a la baja, en línea con el Ratio de Sortino observado.

## Paso de variables binarias a enteras

A continuación se han introducirdo una serie de cambios en el código para manejar de forma más precisa la gestión de una cartera de inversiones:

1.  **Presupuesto y unidades de inversión** :

  -  *Presupuesto y Unidades de Inversión* : Se introducen variables para el presupuesto global. Se calcula el número de unidades invertibles dividiendo el presupuesto total entre la unidad de inversión y se determina el número de bits necesarios utilizando representación binaria.

  -  *Representación de variables con múltiples bits* : Para representar la cantidad invertida en cada activo se utilizan ahora múltiples bits en lugar de una única variable binaria. La varible del problema pasa ahora a definirse como una matriz de bits con dimensiones (n_assets, n_bits), donde cada fila representa un activo y cada columna un bit de la cantidad invertida.

  -  *Construcción del modelo QUBO* : Se construyen los términos H_cuad, H_slack y H_budget considerando los múltiplos de inversión y las combinaciones de bits.

  -  *Preparación y Reporte de Resultados* : Las funciones de preparación y reporte de resultados se adaptan para mostrar las cantidades invertidas por activo y evaluar la validez de las soluciones obtenidas.

  -  *Verificación de Factibilidad* : Se verifica si el retorno mínimo es alcanzable antes de proceder con la construcción del QUBO y la ejecución del solver.

In [76]:
# Calcular los retornos diarios y la matriz de covarianza
retornos = acciones.pct_change().dropna()
cov_matrix = retornos.cov().values

# Calcular los retornos esperados como la media de los retornos diarios
retornos_esperados = retornos.mean().values

# Obtener cantidad de activos N basado en las columnas seleccionadas
n_assets = len(acciones.columns)
asset_names = acciones.columns.tolist()

# Ajustar dinámicamente el umbral de retorno mínimo
retorno_minimo = 0.02

# Presupuesto total disponible para inversión
presupuesto_total = 100
unidad_inversion = 10  # Múltiplo de la inversión
max_unidades = 50 // unidad_inversion  # Máximo número de unidades invertibles por activo
n_bits = int(np.ceil(np.log2(max_unidades + 1)))  # Número de bits necesarios para representar hasta max_unidades

# Función para construir el modelo QUBO
def build_qubo(cov_matrix, n_assets, factor_penalty, retornos, retorno_min_esperado, presupuesto, unidad_inversion, n_bits):
    var_problema = BitArrayShape('x', (n_assets, n_bits))

    # Crear la variable slack
    slack_stop = presupuesto // unidad_inversion
    my_var_slack = VarSlack(name='slack_variable', start=0, step=1, stop=slack_stop, slack_type=SlackType.binary)

    var_shape_set = VarShapeSet(var_problema, my_var_slack)
    BinPol.freeze_var_shape_set(var_shape_set)

    H_cuad = BinPol()

    # Construir H_cuad utilizando la matriz de covarianza
    for i in range(n_assets):
        for j in range(n_assets):
            for k in range(n_bits):
                for l in range(n_bits):
                    w_ij = cov_matrix[i, j] * (2**k) * (2**l) * (unidad_inversion ** 2) # Como los términos (2**k) * (2**l) representan las contribuciones de las diferentes
                    H_cuad.add_term(w_ij, ('x', i, k), ('x', j, l))                     # escalas, por eso se eleva al cuadrado la unidad de inversión, de este modo se mantiene
                                                                                        # la coherencia dimensional con la covarianza
    # Construir H_slack para representar la restricción del retorno mínimo esperado
    H_slack = BinPol()

    for i in range(n_assets):
        for k in range(n_bits):
            H_slack.add_term(retornos_esperados[i] * (2**k) * unidad_inversion, ('x', i, k))

    H_slack.add_slack_variable('slack_variable', factor=-1)
    H_slack.add_term(-retorno_minimo, ())

    # Construir H_budget para representar la restricción del presupuesto
    H_budget = BinPol()

    for i in range(n_assets):
        for k in range(n_bits):
            H_budget.add_term((2**k) * unidad_inversion, ('x', i, k))
    H_budget.add_term(-presupuesto, ())
    H_budget.power(2)

    # Combinar los términos para formar el QUBO
    QUBO = H_cuad + (H_slack + H_budget) * factor_penalty

    return QUBO, H_cuad, H_slack, H_budget

# Coeficiente de penalización para las restricciones
factor_penalty = 1000
QUBO, H_cuad, H_slack, H_budget = build_qubo(cov_matrix, n_assets, factor_penalty, retornos_esperados, retorno_minimo, presupuesto_total, unidad_inversion, n_bits)

# Configurar y ejecutar el solucionador QUBO
solver = QUBOSolverCPU(
    number_iterations=50000,
    number_runs=16,  # Número de ejecuciones en paralelo
    scaling_bit_precision=16,
    auto_tuning=AutoTuning.AUTO_SCALING_AND_SAMPLING
)

solution_list = solver.minimize(QUBO)
print(solution_list.min_solution.configuration)
print()
print(solution_list.solver_times)

# Función para preparar y presentar el resultado
def prep_result(solution_list, H_budget, silent=False):
    solution = solution_list.min_solution  # La mejor solución
    constraint_penalty = H_budget.compute(solution.configuration)
    if not silent:
        print(f'Valor QUBO: {constraint_penalty}')
        print(solution.configuration)
    return constraint_penalty, solution

# Función para reportar la solución
def report(constraint_penalty, solution, asset_names, retorno_min_esperado, unidad_inversion, n_bits, silent=False):
    if not silent:
        if constraint_penalty == 0.0:  # Es cero en las soluciones válidas
            print('Portfolio elegido:')
            selected_assets = []
            for i in range(len(asset_names)):
                cantidad = sum(solution.configuration[i * n_bits + k] * (2**k) for k in range(n_bits))
                if cantidad > 0:
                    selected_assets.append((asset_names[i], cantidad * unidad_inversion))
            print(selected_assets)
        else:
            print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_min_esperado}")

# Función para verificar si el retorno mínimo es alcanzable
def es_factible(retornos, retorno_min_esperado, n_assets):
    retorno_posible_max = sum(sorted(retornos, reverse=True)[:n_assets])
    return retorno_posible_max >= retorno_min_esperado

# Verificar si el retorno mínimo es alcanzable antes de proceder
if not es_factible(retornos_esperados, retorno_minimo, n_assets):
    print(f"No se puede dar solución para el retorno mínimo introducido de {retorno_minimo}")
else:
    # Utilizar las funciones de resultado y reporte
    constraint_penalty, solution = prep_result(solution_list, H_budget)
    report(constraint_penalty, solution, asset_names, retorno_minimo, unidad_inversion, n_bits)


Attention: Downscaling!

********************************************************************************
Scaling qubo, temperature_start, temperature_end and offset_increase_rate
  factor:                            0.00512
********************************************************************************


********************************************************************************
Effective values (including scaling factor)
  temperature_start:                1941.000
  temperature_end:                    68.590
  offset_increase_rate:              269.000
  duration:                            0.002 sec
********************************************************************************

[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]

+--------------+----------------------------+----------------------------+----------------+
| time         | from                       | to                         | duration       |
|--------------+----------------------------+----------------------

In [77]:
import numpy as np
import pandas as pd

def evaluate_portfolio(solution, asset_names, retornos, cov_matrix, annual_risk_free_rate=0.01, retorno_del_mercado=0.10):
    # Asumimos que la solución es una lista de pesos, directamente proporcional a las unidades invertidas
    pesos = np.array([sum(solution[i * n_bits + k] * (2**k) for k in range(n_bits)) * unidad_inversion for i in range(n_assets)])

    # Normalizar los pesos, convirtiendo a float para permitir la división
    pesos = pesos.astype(float) / pesos.sum() # Change the datatype to float to allow division

    # Calcular el retorno y la varianza del portfolio
    retorno_portfolio = np.dot(pesos, retornos.mean()) * 252
    varianza_portfolio = np.dot(pesos.T, np.dot(cov_matrix, pesos))

    # Convertir la tasa libre de riesgo anual a diaria
    daily_risk_free_rate = (1 + annual_risk_free_rate)**(1/252) - 1

    # Calcular la desviación estándar del portfolio
    desviacion_portfolio = np.sqrt(varianza_portfolio)

    # Calcular el ratio de Sharpe
    ratio_sharpe = (retorno_portfolio - daily_risk_free_rate) / desviacion_portfolio if desviacion_portfolio > 0 else 0

    # Identificar retornos negativos y calcular el Ratio de Sortino
    retornos_negativos = retornos[retornos < daily_risk_free_rate]
    varianza_negativa = retornos_negativos.var()
    desviacion_estandar_negativos = np.sqrt(varianza_negativa)
    # Handle the case when 'desviacion_estandar_negativos' is not a scalar
    if desviacion_estandar_negativos.size == 1:
        ratio_sortino = (retorno_portfolio - daily_risk_free_rate) / desviacion_estandar_negativos.item() if desviacion_estandar_negativos.item() > 0 else float('inf')
    else:
        # You may need to adjust this part based on how you want to handle multiple negative returns
        ratio_sortino = (retorno_portfolio - daily_risk_free_rate) / np.sqrt(np.mean(desviacion_estandar_negativos**2)) if np.mean(desviacion_estandar_negativos**2) > 0 else float('inf')

    # Calcular el ratio de Información
    exceso_retornos = retorno_portfolio - retorno_del_mercado
    ratio_informacion = exceso_retornos / desviacion_portfolio if desviacion_portfolio > 0 else 0

    # Calcular la máxima caída
    precios_portfolio = (1 + retornos).cumprod()
    wealth_index = 1000 * precios_portfolio
    previous_peaks = np.maximum.accumulate(wealth_index)
    drawdowns = (wealth_index - previous_peaks) / previous_peaks
    # Extract the minimum drawdown value directly
    max_drawdown = drawdowns.min().min()  # Extract the scalar value directly

    # Imprimir las métricas
    print(f"Retorno del Portfolio: {retorno_portfolio:.4f}")
    print(f"Varianza del Portfolio: {varianza_portfolio:.4f}")
    print(f"Ratio de Sharpe: {ratio_sharpe:.4f}")
    print(f"Ratio de Sortino: {ratio_sortino:.4f}")
    print(f"Ratio de Información: {ratio_informacion:.4f}")
    print(f"Máxima Caída (Drawdown): {max_drawdown:.4f}")

if solution_list.min_solution:
    evaluate_portfolio(solution_list.min_solution.configuration, asset_names, retornos, cov_matrix)

Retorno del Portfolio: 0.0224
Varianza del Portfolio: 0.0105
Ratio de Sharpe: 0.2184
Ratio de Sortino: 0.2221
Ratio de Información: -0.7593
Máxima Caída (Drawdown): -0.5000


# Ejecución en D-Wave

In [55]:
!pip install dimod
!pip install pip-system-certs
!pip install dwave-neal
!pip install dwave-system
!pip install hybrid

In [56]:
# experimental:start
def as_bqm(self) -> 'dimod.BinaryQuadraticModel':
    """
    The polynomial is returned as a :class:`dimod.BinaryQuadraticModel` object.

    :return: qubo as dimod.BinaryQuadraticModel object
    :rtype: dimod.BinaryQuadraticModel
    """

    try:
        import dimod
    except Exception as oops:
        print('\n\n' + (100 * '#'))
        print('pip install dwave-ocean-sdk')
        print((100 * '#') + '\n\n')
        raise oops

    return dimod.BinaryQuadraticModel(
        {i0: self._p1[i0] for i0 in self._p1},
        {(i0, i1): self._p2[i0][i1] for i0 in self._p2 for i1 in self._p2[i0]},
        self._p0,
        dimod.BINARY)


In [57]:
bqm_problem=QUBO.as_bqm()

In [58]:
os.environ['DWAVE_API_TOKEN']='DEV-8118e94d4ee0db7e32f9198eb7e865c8a30acfa5'

## Solver Híbrido

In [59]:
from dimod.generators import and_gate
from dwave.system import LeapHybridSampler

sampler = LeapHybridSampler()
answer = sampler.sample(bqm_problem)
print(answer)

   0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15       energy num_oc.
0  1  1  0  0  0  0  0  0  0  1  1  1  1  1  1  1 -9323.986529       1
['BINARY', 1 rows, 1 samples, 16 variables]


In [60]:
for datum in answer.data(['sample', 'energy']):
   print(datum.sample, datum.energy)

{0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1} -9323.986529313726


In [61]:
solution_dwave=list(datum.sample.values())

In [62]:
print( "QUBO     = %10.6f" % (QUBO.compute(solution_dwave)) )
print( "H_cuad   = %10.6f" % (H_cuad.compute(solution_dwave)) )
print( "H_slack  = %10.6f" % (H_slack.compute(solution_dwave)) )
print( "H_budget = %10.6f" % (H_budget.compute(solution_dwave)) )

QUBO     = -9323.986529
H_cuad   =  98.792125
H_slack  =  -9.422779
H_budget =   0.000000


In [81]:
def as_bqm(self) -> 'dimod.BinaryQuadraticModel':
    """
    Convertir el polinomio a un objeto dimod.BinaryQuadraticModel.
    """
    try:
        import dimod
    except ImportError as error:
        print('\n\n' + (100 * '#'))
        print('Please install dwave-ocean-sdk')
        print((100 * '#') + '\n\n')
        raise error

    return dimod.BinaryQuadraticModel(
        {i0: self._p1[i0] for i0 in self._p1},
        {(i0, i1): self._p2[i0][i1] for i0 in self._p2 for i1 in self._p2[i0]},
        self._p0,
        dimod.BINARY)

# Convierte el QUBO a un modelo BQM
bqm_problem = QUBO.as_bqm()

# Configuración del sampler de D-Wave
import os
from dwave.system import LeapHybridSampler

os.environ['DWAVE_API_TOKEN'] = 'DEV-8118e94d4ee0db7e32f9198eb7e865c8a30acfa5'
sampler = LeapHybridSampler()

# Muestreo del problema BQM
answer = sampler.sample(bqm_problem)

# Análisis de la solución
for datum in answer.data(['sample', 'energy']):
    print("Configuración de solución: ", datum.sample)
    print("Energía: ", datum.energy)



Configuración de solución:  {0: 1, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1}
Energía:  -9323.986529313726


## Solver Cuántico

In [84]:
import time
from dwave.system.composites import EmbeddingComposite
from dwave.system.samplers import DWaveSampler

# Mide el tiempo de muestreo
sample_time = time.time()

# Establece un sampler QPU con incrustación
sampler = EmbeddingComposite(DWaveSampler())

# Número de lecturas
num_reads = 2000

# Muestrea el problema BQM
sampleset = sampler.sample(bqm_problem,
                           num_reads=num_reads,
                           label='Purely Quantum Exec')

# Analiza la solución con la menor energía
for datum in sampleset.lowest().data(['sample', 'energy']):
    print("Configuración de solución: ", datum.sample)
    print("Energía: ", datum.energy)

# No se realizan cálculos locales de QUBO, H_cuad, H_slack, o H_budget


Configuración de solución:  {0: 0, 1: 1, 2: 0, 3: 0, 4: 0, 5: 0, 6: 1, 7: 0, 8: 0, 9: 1, 10: 1, 11: 1, 12: 1, 13: 1, 14: 1, 15: 1}
Energía:  -9043.523469923297
